# LangChain: Models, Prompts and Output Parsers

### Outline
- Direct API calls to OpenAI
- API calls through LangChain:
    - Prompts
    - Models
    - Output parsers

In [5]:
#!pip install python-dotenv
#!pip install openai

In [6]:
import os
from openai import OpenAI

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])

## Chat API : OpenAI
Let's start with a direct API calls to OpenAI.

In [54]:
def get_completion(prompt, model="gpt-4o"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, 
    )
    return response.choices[0].message.content

In [55]:
get_completion("What is 1+1?")

'1 + 1 equals 2.'

In [9]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [10]:
style = """American English \
in a calm and respectful tone
"""

In [11]:
prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

Translate the text that is delimited by triple backticks 
into a style that is American English in a calm and respectful tone
.
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



In [12]:
response = get_completion(prompt)

In [13]:
response

"Arrgh, I'm pretty upset that my blender lid flew off and splattered my kitchen walls with smoothie! And to top it off, the warranty doesn't cover the cost of cleaning up my kitchen. I could really use your help right now, friend."

## Chat API : LangChain
Let's try how we can do the same thing using LangChain

In [14]:
#!pip install --upgrade langchain

### Model 

In [15]:
from langchain_openai import ChatOpenAI

In [16]:
# To control the randomness and creativity of the generated
# text by an LLM, use temperature = 0.0
chat = ChatOpenAI(temperature=0.0)
chat

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x7ff8f5ecbd30>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x7ff8f5e88e20>, root_client=<openai.OpenAI object at 0x7ff8f5ecba90>, root_async_client=<openai.AsyncOpenAI object at 0x7ff8f5e886d0>, temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True)

### Prompt template

In [17]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [18]:
from langchain_core.prompts import ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_template(template_string)

In [19]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], input_types={}, partial_variables={}, template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n')

In [20]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [21]:
customer_style = """American English \
in a calm and respectful tone
"""

In [22]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [23]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [24]:
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>


In [25]:
# Prompt expected by the user 
print(customer_messages[0])

content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n" additional_kwargs={} response_metadata={}


In [26]:
# Call the LLM to translate to the style of the customer message
# Reference: chat = ChatOpenAI(temperature=0.0)
customer_response = chat(customer_messages)

/tmp/ipykernel_320903/2955040366.py:3: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  customer_response = chat(customer_messages)


In [27]:
print(customer_response.content)

I'm really frustrated that my blender lid flew off and splattered my kitchen walls with smoothie! And to make matters worse, the warranty doesn't cover the cost of cleaning up my kitchen. I could really use your help right now, friend.


In [28]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [29]:
service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

In [30]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
```



In [31]:
service_response = chat(service_messages)
print(service_response.content)

Ahoy there, valued customer! Regrettably, the warranty be not coverin' the cost o' cleanin' yer galley due to yer own negligence. Ye see, 'twas yer own doin' that ye forgot to secure the lid afore startin' the blender. 'Tis a tough break, indeed! Fare thee well, matey!


## Output Parsers
Let's start with defining how we would like the LLM output to look like:

In [32]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [33]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [34]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'), additional_kwargs={})]


In [35]:
messages = prompt_template.format_messages(text=customer_review)
chat = ChatOpenAI(temperature=0.0)
response = chat(messages)
print(response.content)

{
    "gift": true,
    "delivery_days": 2,
    "price_value": "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
}


In [36]:
type(response.content)

str

In [37]:
# You will get an error by running this line of code 
# because'gift' is not a dictionary
# 'gift' is a string
response.content.get('gift')

AttributeError: 'str' object has no attribute 'get'

## Parse the LLM output string into a Python dictionary

In [40]:
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field

In [41]:
# Define the output schema using Pydantic
class ReviewResponse(BaseModel):
    gift: bool = Field(description="Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.")
    delivery_days: int = Field(description="How many days did it take for the product to arrive? If this information is not found, output -1.")
    price_value: str = Field(description="Extract any sentences about the value or price, and output them as a comma separated Python list.")

In [56]:
output_parser = JsonOutputParser(pydantic_object=ReviewResponse)

In [57]:
print(output_parser)

pydantic_object=<class '__main__.ReviewResponse'>


In [62]:
format_instructions = output_parser.get_format_instructions()

In [63]:
print(format_instructions)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"gift": {"description": "Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.", "title": "Gift", "type": "boolean"}, "delivery_days": {"description": "How many days did it take for the product to arrive? If this information is not found, output -1.", "title": "Delivery Days", "type": "integer"}, "price_value": {"description": "Extract any sentences about the value or price, and output them as a comma separated Python list.", "title": "Price Value", "type": "string"}}, "required": ["gif

In [60]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)

In [61]:
print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties

In [48]:
response = chat(messages)

In [49]:
print(response.content)

{
  "gift": false,
  "delivery_days": 2,
  "price_value": "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
}


In [64]:
output_dict = output_parser.parse(response.content)

In [65]:
output_dict

{'gift': False,
 'delivery_days': 2,
 'price_value': "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."}

In [66]:
type(output_dict)

dict

In [67]:
output_dict.get('delivery_days')

2